In [15]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [16]:
import nbodykit.lab
from nbodykit.lab import *
from nbodykit import style, setup_logging

import numpy as np
import MAS_library as MASL
import Pk_library as PKL
import matplotlib.pyplot as plt
plt.style.use(style.notebook)

import sys, pickle, time, os

# nbodykit tool to read custom (non-standard) catalogue file type
from nbodykit.io.base import FileType
# nbodykit tool to creae custom subclass od CatalogSource
from nbodykit.source.catalog.file import FileCatalogFactory
# nbodykit cosmology parameters initialization
from nbodykit.lab import cosmology
# nbodykit other way to create custom catalog from array
from nbodykit.source.catalog import ArrayCatalog

from readfof import FoF_catalog
from torch import flatten, from_numpy

In [17]:
setup_logging()

In [18]:
order_folders = {
    'fiducial'  : 0,
    'h_m'       : 1,
    'h_p'       : 2,
    'Mnu_p'     : 3,
    'Mnu_pp'    : 4,
    'Mnu_ppp'   : 5,
    'ns_m'      : 6,
    'ns_p'      : 7,
    'Ob2_m'     : 8,
    'Ob2_p'     : 9,
    'Om_m'      : 10,
    'Om_p'      : 11,
    's8_m'      : 12,
    's8_p'      : 13,
    'w_m'       : 14,
    'w_p'       : 15,
    'zeldovich' : 16
}

cosmological_pars = {
    'Om'  : 0,
    'Ob'  : 1,
    'h'   : 2,
    'ns'  : 3,
    's8'  : 4,
    'w'   : 5,
    'Mnu' : 6
}

order_dimension = {
    'Om'  : 0,
    'Ob'  : 1,
    'Ob2' : 1,
    'h'   : 2,
    'ns'  : 3,
    's8'  : 4,
    'w'   : 5,
    'Mnu' : 6
}

boolean = {
    'True' : 1,
    'TRUE' : 1,
    'true' : 1,
    'False' : 0,
    'FALSE' : 0,
    'false' : 0
 }

COSMOPAR = {
#                   | Om   | Ob   |   h   |  n_s  | s_8 | Mnu | w |

    'fiducial' :    [0.3175, 0.049, 0.6711, 0.9624, 0.834, 0, -1],
    'zeldovich':    [0.3175, 0.049, 0.6711, 0.9624, 0.834, 0, -1],
    
    'Mnu_p' :       [0.3175, 0.049, 0.6711, 0.9624, 0.834, 0.1, -1],
    'Mnu_pp' :      [0.3175, 0.049, 0.6711, 0.9624, 0.834, 0.2, -1],
    'Mnu_ppp' :     [0.3175, 0.049, 0.6711, 0.9624, 0.834, 0.4, -1],
    
    'h_m' :         [0.3175, 0.049, 0.6511, 0.9624, 0.834, 0, -1],
    'h_p' :         [0.3175, 0.049, 0.6911, 0.9624, 0.834, 0, -1],
    
    'ns_m' :        [0.3175, 0.049, 0.6711, 0.9424, 0.834, 0, -1],
    'ns_p' :        [0.3175, 0.049, 0.6711, 0.9824, 0.834, 0, -1],
    
    'Ob_m' :        [0.3175, 0.048, 0.6711, 0.9624, 0.834, 0, -1],
    'Ob_p' :        [0.3175, 0.050, 0.6711, 0.9624, 0.834, 0, -1],
    'Ob2_m' :       [0.3175, 0.047, 0.6711, 0.9624, 0.834, 0, -1],
    'Ob2_p' :       [0.3175, 0.051, 0.6711, 0.9624, 0.834, 0, -1],
    
    'Om_m' :        [0.3075, 0.049, 0.6711, 0.9624, 0.834, 0, -1],
    'Om_p' :        [0.3275, 0.049, 0.6711, 0.9624, 0.834, 0, -1],
    
    's8_m' :        [0.3175, 0.049, 0.6711, 0.9624, 0.819, 0, -1],
    's8_p' :        [0.3175, 0.049, 0.6711, 0.9624, 0.849, 0, -1],
    
    'w_m' :         [0.3175, 0.049, 0.6711, 0.9624, 0.834, 0, -0.95],
    'w_p' :         [0.3175, 0.049, 0.6711, 0.9624, 0.834, 0, -1.05]
}
"""
Dictionary to assegnate to a cosmology its parameters
Ordered pairing cosmologies and the first one in with _m, the second one _p
"""

VarCosmoPar = {
    'd_h'  : 0.02,
    'd_ns' : 0.02,
    'd_Ob' : 0.001,
    'd_Ob2': 0.002,
    'd_Om' : 0.01,
    'd_s8' : 0.015,
    'd_w'  : -0.05
}
"""Values of deviations from fiducial cosmological parameters.
"""

fiducial_vals = {
    'Ob'  : 0.3175,
    'Ob2' : 0.3175,
    'Om'  : 0.049,
    'h'   : 0.6711,
    'n_s' : 0.9624,
    'ns'  : 0.9624,
    's_8' : 0.834,
    's8'  : 0.834,
    'Mnu' : 0,
    'w'   : -1
}
"""Values of fiducial cosmological parameters.
"""

'Values of fiducial cosmological parameters.\n'

In [19]:
def info_name(name):
    """Obtain realization information from namefile"""
    # assert type(name) == str
    print("TYPE: ", type(name))
    # info = name.split('_')
    info = name.split('_')[-3:]
    print("TYPE: ", info, "\n", type(info))
    info[2] = info[2].replace(".wst", "")
    N_hgrid = info[0]
    N_WSTgrid = info[1]
    n_realiz = info[2].replace(".wst", "")
    
    return [int(N_hgrid), int(N_WSTgrid), int(n_realiz)]

def cosmo_parser(name):
    """Obtain cosmology from .wst file"""
    info = name.split('_')
    if info[0] == "fiducial":
        return info[0]
    elif info[0] == "zeldovich":
        return info[0]
    else:
        return info[0] + "_" + info[1]

In [20]:
def PacMan(x, d = np.array((0, 0, 1000)) ):
   """Returns a number x in the interval [0; d]"""
   if 0 <= x[2] <= d[2]:
      return x
   elif x[2] > d[2]:
      return PacMan(x - d)
   elif x[2] < 0:
      return PacMan(x + d) 

In [21]:
def PowerSpec_Real(dens, hlength, filename):
   filename = filename.replace(".wst", ".pk")
   name = '/home/riccardo/Pk-files/'+filename
   Pk = PKL.Pk(dens, hlength, axis=0, MAS = 'CIC', verbose=False)
   with open(name.replace("_Pk_", "_k3D_"), 'ab') as file:
      pickle.dump(Pk.k3D, file)
   with open(name.replace("_Pk_", "_Nmodes3D_"), 'ab') as file:
      pickle.dump(Pk.Nmodes3D, file)
   with open(name.replace("_Pk_", "_kpar_"), 'ab') as file:
      pickle.dump(Pk.kpar, file)
   with open(name.replace("_Pk_", "_kper_"), 'ab') as file:
      pickle.dump(Pk.kper, file)
   with open(name.replace("_Pk_", "_Pk_"), 'ab') as file:
      pickle.dump(Pk.Pk, file)

def PowerSpec_RSD(dens, hlength, filename):
   filename = filename.replace(".wst", ".pk")
   Pk = PKL.Pk(dens, hlength, axis=2, MAS = 'CIC', verbose=False)
   name = '/home/riccardo/Pk-files/'+filename
   with open(name.replace("_Pk_", "_k3D_"), 'ab') as file:
      pickle.dump(Pk.k3D, file)
   with open(name.replace("_Pk_", "_Nmodes3D_"), 'ab') as file:
      pickle.dump(Pk.Nmodes3D, file)
   with open(name.replace("_Pk_", "_kpar_"), 'ab') as file:
      pickle.dump(Pk.kpar, file)
   with open(name.replace("_Pk_", "_kper_"), 'ab') as file:
      pickle.dump(Pk.kper, file)
   with open(name.replace("_Pk_", "_Pk_"), 'ab') as file:
      pickle.dump(Pk.Pk, file)

In [22]:
def HaloWST_one_f_MASL(filename, snapdir, i, snapnum=2, N_grid=256, hlength=1000, j=4, l=4):
   """Funcion that evaluates Scattering Transform coefficients of first and second order,
   using a halo database from Quijote simulations.
   NB: this function uses `MAS_library.MA()` to generate density matrix, not `myCIC`.

   Arguments:
   - `filename`: name of the file conaining WST coefficients of first and second order
   - `snapdir` : the tree to the directory containing the datas, STOP before '/groups_';
   - `snapnum` : indicates the choosen redshift (def: 2)
   - `N_hgrid` : number of cells to divide the halos catalogue (def: 256)
   - `hlength` : dimension of the cubic simulation (def: 1000 Mpc/h)
   - `N_WSTgrid` : number of cells to divide the density field, generated by myCIC,
      to calculate WST coefficients (def: 256)
   - `j` : coefficient for scattering transform evaluation (def: 4)
   - `l` : coefficient for scattering transform evaluation (def: 4)
   - `i` : oprional, svariable for print execution time, takes value from iterable object of for loop

   Returns:
   - prints fist and second order scattering coefficients into a single files as 1D array.
   """
   
   z_dict = {4:0.0, 3:0.5, 2:1.0, 1:2.0, 0:3.0}
   redshift = z_dict[snapnum]
   strt = time.time()
   H_0 = 100 * COSMOPAR[cosmo_parser(filename)][2]

   print(i," - Reading datas", flush = True) 
   datas = FoF_catalog(snapdir, snapnum, read_IDs=False)
   print("  Data file readed -> variables assignement", flush = True)
   
   # ======== Reading datas ==========================================================================================
   pos_h = (datas.GroupPos/1e3).astype(np.float32)                     # positions in Mpc/h
   mass = (datas.GroupMass * 1e10).astype(np.float32)                  # masses in M_sun/
   vel = (datas.GroupVel*(1.0+redshift)).astype(np.float32)            # Halo peculiar velocities in km/s
   del datas

   # if len(mass_M) > 0:
   START = time.time()
   dens = np.zeros((N_grid, N_grid, N_grid), dtype=np.float32)
   MASL.MA(pos_h, dens, hlength, 'CIC', W = mass)
   dens /= np.mean(dens, dtype=np.float32)
   dens -= 1.0
   # WST eval
   # Sx = S.scattering(from_numpy(dens))
   # with open('/home/riccardo/WST-files/'+filename.replace("_coefficients_", "_coefficients_M_Ns_"), 'ab') as file:
   #    pickle.dump(flatten(Sx, start_dim=0).cpu().detach().numpy(), file)

   # Pk eval
   PowerSpec_Real(dens, hlength, filename.replace("_coefficients_", "_Pk_M_Ns_"))
   PowerSpec_RSD(dens, hlength, filename.replace("_coefficients_", "_Pk_M_RDs_"))

   print("  completed M_Ns in: ", round((time.time()-START)/60, 3), " min", flush = True)

   del dens#, Sx

   START = time.time()
   dens = np.zeros((N_grid, N_grid, N_grid), dtype=np.float32)
   for i in range(len(vel)):
      pos_h[i] = PacMan(pos_h[i] + (1+redshift) * np.array( ([0, 0, vel[i][2]/H_0]) ) )
   MASL.MA(pos_h, dens, hlength, 'CIC', W = mass)
   dens /= np.mean(dens, dtype=np.float32)
   dens -= 1.0
   # WST eval
   # Sx = S.scattering(from_numpy(dens))   
   # with open('/home/riccardo/WST-files/'+filename.replace("_coefficients_", "_coefficients_M_RDs_"), 'ab') as file:
   #    pickle.dump(flatten(Sx, start_dim=0).cpu().detach().numpy(), file)
   # PowerSpec_RSD(dens, hlength, filename.replace("_coefficients_", "_Pk_M_RDs_"))
   print("  completed M_Rs in: ", round((time.time()-START)/60, 3), " min", flush = True)

   print("  The files' been written\n    TIME LAP: ", round((time.time()-strt)/60, 3), " mins\n", flush = True)

In [23]:
def CALCULUS(S, N_grid = 256, n_realiz = -1, Ff = 'fiducial', root='/workplace/riccardo/Halos/', sig = "s08", qu = "q08"):
   """Evaluates WST coefficients and print them in one/two files (this is an option) for given folders
   using `HaloWST_one_f_MASL`
   """

   if n_realiz > 0:
      num = n_realiz
      in_realizations = os.listdir(root+Ff)[0:num]
   elif n_realiz < 0:
      in_realizations = os.listdir(root+Ff)
      num = len(in_realizations)
   else:
      assert False, "No realization to analize!"
    
   filename = Ff+'_coefficients_' +sig + '_' + qu + '_' +str(N_grid)+"_"+str(num)+'.wst'

   # delete existing file, want a new one (not extending it)
   if not os.path.exists('/home/riccardo/WST-files/'+filename):
      # os.remove('/home/riccardo/WST-files/'+filename)
      for i in range(len(in_realizations)):
         snapdir = root + Ff + '/' + in_realizations[i]
         HaloWST_one_f_MASL(filename = filename, snapdir = snapdir, N_grid = N_grid, i=i)
   else:
      print("WARNING: ", filename, "already exists")

In [24]:
N_grid = 256
hlength = 1000

snapnum = 2
filename = "fiducial"+'_coefficients_' +"08" + '_' + "08" + '_' +str(N_grid)+"_"+str(1000)+'.wst'
snapdir = "/media/fuffolo97/HDD1/UNI/Tesi/halos/" + cosmo_parser(filename) + "/0"

z_dict = {4:0.0, 3:0.5, 2:1.0, 1:2.0, 0:3.0}
redshift = z_dict[snapnum]
strt = time.time()
H_0 = 100 * COSMOPAR["fiducial"][2]

datas = FoF_catalog(snapdir, snapnum, read_IDs=False)

In [25]:
pos_h = (datas.GroupPos/1e3).astype(np.float32)                     # positions in Mpc/h
mass = (datas.GroupMass * 1e10).astype(np.float32)                  # masses in M_sun/
vel = (datas.GroupVel*(1.0+redshift)).astype(np.float32)            # Halo peculiar velocities in km/s

In [26]:
cosmo_nbk = cosmology.Cosmology(h=fiducial_vals["h"],
                                sigma8=fiducial_vals["s8"],
                                #Omega0_b=fiducial_vals["Ob"],
                                Omega0_cdm = fiducial_vals["Om"],
                                n_s=fiducial_vals["ns"],
                                # m_ncdm=fiducial_vals["Mnu"]
                                )

BoxSize = 1000.
line_of_sight = [0, 0, 1]

## Method (A)

In [27]:
data_nbk = np.empty(len(mass), dtype=[('Position', ('f8', 3)), ('RSDPosition', ('f8', 3)),('Velocity', ('f8', 3)), ('Mass', 'f8')])
data_nbk["Position"] = pos_h
data_nbk["Velocity"] = vel
data_nbk["Mass"] = mass
data_nbk['RSDPosition'] = data_nbk['Position'] + data_nbk['Velocity'] * line_of_sight
np.save("./npy-example_A.npy", data_nbk)

In [28]:
class QuijFile(FileType):
    """A file-like object to read 'xxx.0' Quijote catalog"""
    def __init__(self, path, Nmesh=256):
        self.path = path
        self.attrs = {}
        # load the data and set size and dtype
        self._data = np.load(self.path)
        self.size  = len(self._data)  # total size
        self.dtype = self._data.dtype # data dtype

    def read(self, columns, start, stop, step=1):
        return self._data[start:stop:step]
    
QuijCatalog = FileCatalogFactory('QuijCatalog', QuijFile)

In [29]:
f = QuijCatalog("./npy-example_A.npy")
mesh = f.to_mesh(resampler='cic', Nmesh=256, compensated=True, position='Position', weight="Mass", BoxSize=BoxSize)

[ 000014.78 ]   0: 09-27 14:18  CatalogSource   INFO     Extra arguments to FileType: () {}


In [30]:
FFTPower(mesh, mode='2d', dk=0.005, kmin=0.01, Nmu=5, los=[0,0,1], poles=[0,2,4])

[ 000014.85 ]   0: 09-27 14:18  CatalogMesh     INFO     Chunk 0 ~ 4194304 / 195524 


ValueError: error trying to view slice as a single numpy array: cannot reshape array of size 10 into shape (3)

In [ ]:
r_3d = FFTPower(mesh, mode='1d', dk=0.005, kmin=0.01)

## Method (B)

In [ ]:
dtype_B = [("Position", (np.float32, 3)), ("Velocity", (np.float32, 3)), ("Mass", np.float32)]

In [ ]:
data_B = np.column_stack((pos_h, vel, mass))

In [ ]:
custom_catalog = np.array(data_B, dtype=dtype_B)

In [ ]:
custom_catalog = ArrayCatalog(custom_catalog)

In [ ]:
mesh_B = custom_catalog.to_mesh(resampler='cic', Nmesh=256, compensated=True, position='Position', weight="Mass", BoxSize=BoxSize)

In [ ]:
r_3d = FFTPower(mesh_B, mode='1d', dk=0.005, kmin=0.01)

In [ ]:
print("pos_h shape:", pos_h.shape)
print("vel shape:", vel.shape)
print("mass shape:", mass.shape)

pos_h shape: (195524, 3)
vel shape: (195524, 3)
mass shape: (195524,)


## Method (C)

In [ ]:
Plin = cosmology.LinearPower(cosmo_nbk, 1)